In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm

2023-09-06 23:17:12.794065: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-06 23:17:14.602784: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
class NN(pl.LightningModule):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1        = nn.Linear(input_size, 50)
        self.fc2        = nn.Linear(50, num_classes)
        self.loss_fn    = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

    # # Old way
    # # We don't need to perform zero_grad() and loss.backward() manually
    # def training_step(self, batch, batch_idx):
    #     x, y    = batch
    #     # Flatten image
    #     x       = x.reshape(x.shape[0], -1)
    #     logits  = self.forward(x)
    #     loss    = F.cross_entropy(logits, y)
    #     return loss

    # def validation_step(self, batch, batch_idx):
    #     x, y    = batch
    #     # Flatten image
    #     x       = x.reshape(x.shape[0], -1)
    #     logits  = self.forward(x)
    #     loss    = F.cross_entropy(logits, y)
    #     return loss

    # def test_step(self, batch, batch_idx):
    #     x, y    = batch
    #     # Flatten image
    #     x       = x.reshape(x.shape[0], -1)
    #     logits  = self.forward(x)
    #     loss    = F.cross_entropy(logits, y)
    #     return loss

    # We can use _common_step() to reduce code duplication
    def _common_step(self, batch, batch_idx):
        x, y    = batch
        # Flatten image
        x       = x.reshape(x.shape[0], -1)
        logits  = self.forward(x)
        loss    = F.cross_entropy(logits, y)
        return loss, logits, y

    def training_step(self, batch, batch_idx):
        loss, logits, y = self._common_step(batch, batch_idx)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, logits, y = self._common_step(batch, batch_idx)
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        loss, logits, y = self._common_step(batch, batch_idx)
        self.log('test_loss', loss)
        return loss


    def predict_step(self, batch, batch_idx):
        x, y    = batch
        # Flatten image
        x       = x.reshape(x.shape[0], -1)
        logits  = self.forward(x)
        preds   = torch.argmax(logits, dim=1)
        return preds

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-3)

    